# コード5-1：MCMCを用いたベイズ推定の実装

『ウイルス感染の数理モデルとシミュレーション ーデータを定量的に理解するー』

MCMCを用いたベイズ推定の実装の例をまとめる。

## ライブラリ、パッケージ（モジュール）の読み込み

In [1]:
## 必要があればインストール
# !pip3 install matplotlib numpy gillespy2 time

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gillespy2
import time ## 実行時間測定

## MCMCを用いたベイズ推定

28 Ito, Y., et al., Dynamics of HIV-1 coinfection in different susceptible target cell populations during cell-free infection. J Theor Biol, 2018. 455: p. 39-46.

式(3.3)


pystanパッケージを用いて行う

In [2]:
## データセット
## 論文からとるか擬似データを使うか

In [ ]:
## 推定

### 常微分方程式

ウイルスダイナミクスのような常微分方程式系によるシミュレーションと併せたベイズ推定手法を紹介する。

式（2.12）

$$
\begin{align}
\frac{d T \left( t \right)}{dt} &= L(t) - \beta T(t) V(t) \\
\frac{d I \left( t \right)}{dt} &= \beta T(t) V(t) - \delta I(t) \\
\frac{d V \left( t \right)}{dt} &= p I(t) - c V(t)
\end{align}
$$

について
$L(t) = \lambda - d T(t), \lambda = 0, d = 0.057, \beta = 8.61 \times 10^{-11}, \delta = 0.057 + 1.75, p = 3.26 \times 10^4, c = 1.93 + 0.039, T(0) = 6.46 \times 10^6, I(0) = 10^{-1}, V(0) = 5.0 \times 10^4$

In [4]:
## 擬似データの生成

In [5]:
## 推定